In [1]:
import numpy as np

In [5]:
from keras.models import Sequential,Model
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D, Deconvolution2D, Cropping2D
from keras.layers import Input, Add, Dropout, Permute, add

In [25]:
def Convblock(channel_dimension, block_no, no_of_convs) :
    Layers = []
    for i in range(no_of_convs) :
        
        Conv_name = "CONV_"+str(block_no)+"_"+str(i+1)
        
        
        Layers.append(Convolution2D(channel_dimension,kernel_size = (3,3),padding = "same",activation = "relu",name = Conv_name))
    
    Max_pooling_name = "MaxPool_"+str(block_no)
    Layers.append(MaxPooling2D(pool_size=(2, 2), strides=(2, 2),name = Max_pooling_name))
    
    return Layers

In [28]:
def FCN_16_helper(image_size):
    model = Sequential()
    model.add(Permute((1,2,3),input_shape = (image_size,image_size,3)))
    
    
    for l in Convblock(64,1,2) :
        model.add(l)
    
    for l in Convblock(128,2,2):
        model.add(l)
    
    for l in Convblock(256,3,3):
        model.add(l)
    
    for l in Convblock(512,4,3):
        model.add(l)
    
    for l in Convblock(512,5,3):
        model.add(l)
    
    model.add(Convolution2D(4096,kernel_size=(7,7),padding = "same",activation = "relu",name = "Fully_connected_1"))
    
    model.add(Convolution2D(4096,kernel_size=(1,1),padding = "same",activation = "relu",name = "Fully_connected_2"))
    
    model.add(Convolution2D(21,kernel_size=(1,1),padding="same",activation="relu",name = "Score"))
    
    
    model.add(Deconvolution2D(21,kernel_size=(4,4),strides = (2,2),padding = "valid",activation=None,name = "Upconv1"))
    
    model.add(Cropping2D(cropping=((0,2),(0,2))))
    return model

output = FCN_16(512)
print(len(output.layers))
print(output.layers[14])
print output.summary()
    
    

24
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_12 (Permute)         (None, 512, 512, 3)       0         
_________________________________________________________________
CONV_1_1 (Conv2D)            (None, 512, 512, 64)      1792      
_________________________________________________________________
CONV_1_2 (Conv2D)            (None, 512, 512, 64)      36928     
_________________________________________________________________
MaxPool_1 (MaxPooling2D)     (None, 256, 256, 64)      0         
_________________________________________________________________
CONV_2_1 (Conv2D)            (None, 256, 256, 128)     73856     
_________________________________________________________________
CONV_2_2 (Conv2D)            (None, 256, 256, 128)     147584    
_________________________________________________________________
MaxPool_2 (MaxPooling2D)     (None, 128, 128, 128)     0         
_______

In [31]:
def FCN_16(image_size) :
    fcn_16 = FCN_16_helper(512)
    
    skip_con = Convolution2D(21,kernel_size=(1,1),padding = "same",activation=None, name = "Skip_con")
    
    Summed = add(inputs = [skip_con(fcn_16.layers[14].output),fcn_16.layers[-1].output])
    
    Up = Deconvolution2D(21,kernel_size=(32,32),strides = (16,16),padding = "valid",activation = None,name = "Upsample")
    
    crop = Cropping2D(cropping = ((0,16),(0,16)))
    return Model(fcn_16.input, crop(Up(Summed)))

output = FCN_16(512)

print(output.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
permute_15_input (InputLayer)    (None, 512, 512, 3)   0                                            
____________________________________________________________________________________________________
permute_15 (Permute)             (None, 512, 512, 3)   0           permute_15_input[0][0]           
____________________________________________________________________________________________________
CONV_1_1 (Conv2D)                (None, 512, 512, 64)  1792        permute_15[0][0]                 
____________________________________________________________________________________________________
CONV_1_2 (Conv2D)                (None, 512, 512, 64)  36928       CONV_1_1[0][0]                   
___________________________________________________________________________________________